In [1]:
import numpy as np
import pandas as pd
import datetime
from sklearn.utils import resample
from typing import Dict
from transformers.columnSelectorTransformer import ColumnSelectorTransformer
from transformers.woeTransformer import WOETransformer
from transformers.binningTransformer import BinningTransformer

In [2]:
from bins import bins

In [3]:
dataset = pd.read_csv('../files/credit_risk_data_v2.csv', low_memory=False)

In [4]:
cols_to_keep = ['loan_amnt', 'funded_amnt', 'funded_amnt_inv', 'term', 'int_rate',
       'installment', 'grade', 'sub_grade', 'emp_length', 'home_ownership',
       'annual_inc', 'verification_status', 'pymnt_plan', 'purpose',
       'addr_state', 'dti', 'delinq_2yrs', 'inq_last_6mths',
       'open_acc', 'pub_rec', 'revol_bal',
       'revol_util', 'total_acc', 'initial_list_status', 'out_prncp',
       'out_prncp_inv', 'total_pymnt', 'total_pymnt_inv', 'total_rec_prncp',
       'total_rec_int', 'total_rec_late_fee', 'recoveries',
       'collection_recovery_fee', 'last_pymnt_amnt',
       'collections_12_mths_ex_med', 'acc_now_delinq', 'tot_coll_amt',
       'tot_cur_bal', 'total_rev_hi_lim', 'status']

In [5]:
column_t = ColumnSelectorTransformer(columns=cols_to_keep)
#binning_t = ...
#woe_t = ...

In [6]:
dataset_c = column_t.transform(dataset)
dataset_c.head(25)

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_length,home_ownership,...,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_amnt,collections_12_mths_ex_med,acc_now_delinq,tot_coll_amt,tot_cur_bal,total_rev_hi_lim,status
0,5000,5000,4975.00000,36 months,10.65,162.87,B,B2,10+ years,RENT,...,0.00,0.00,0.0000,171.62,0.0,0.0,NaN,NaN,NaN,0
1,2500,2500,2500.00000,60 months,15.27,59.83,C,C4,< 1 year,RENT,...,0.00,117.08,1.1100,119.66,0.0,0.0,NaN,NaN,NaN,1
2,2400,2400,2400.00000,36 months,15.96,84.33,C,C5,10+ years,RENT,...,0.00,0.00,0.0000,649.91,0.0,0.0,NaN,NaN,NaN,0
3,10000,10000,10000.00000,36 months,13.49,339.31,C,C1,10+ years,RENT,...,16.97,0.00,0.0000,357.48,0.0,0.0,NaN,NaN,NaN,0
4,3000,3000,3000.00000,60 months,12.69,67.79,B,B5,1 year,RENT,...,0.00,0.00,0.0000,67.79,0.0,0.0,NaN,NaN,NaN,0
5,5000,5000,5000.00000,36 months,7.90,156.46,A,A4,3 years,RENT,...,0.00,0.00,0.0000,161.03,0.0,0.0,NaN,NaN,NaN,0
6,7000,7000,7000.00000,60 months,15.96,170.08,C,C5,8 years,RENT,...,0.00,0.00,0.0000,170.08,0.0,0.0,NaN,NaN,NaN,0
7,3000,3000,3000.00000,36 months,18.64,109.43,E,E1,9 years,RENT,...,0.00,0.00,0.0000,111.34,0.0,0.0,NaN,NaN,NaN,0
8,5600,5600,5600.00000,60 months,21.28,152.39,F,F2,4 years,OWN,...,0.00,189.06,2.0900,152.39,0.0,0.0,NaN,NaN,NaN,1
9,5375,5375,5350.00000,60 months,12.69,121.45,B,B5,< 1 year,RENT,...,0.00,269.29,2.5200,121.45,0.0,0.0,NaN,NaN,NaN,1


In [7]:
dataset_c = column_t.clean(X=dataset_c)

In [8]:
dataset_c.head(20)

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_length,home_ownership,...,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_amnt,collections_12_mths_ex_med,acc_now_delinq,tot_coll_amt,tot_cur_bal,total_rev_hi_lim,status
42535,27050,27050,27050.0,36 months,10.99,885.46,B,B2,10+ years,OWN,...,0.0,0.0,0.0,885.46,0.0,0.0,0.0,114834.0,59900.0,0
42536,9750,9750,9750.0,36 months,13.98,333.14,C,C1,1 year,RENT,...,0.0,0.0,0.0,333.14,0.0,0.0,0.0,14123.0,15100.0,0
42537,12000,12000,12000.0,36 months,6.62,368.45,A,A2,10+ years,MORTGAGE,...,0.0,0.0,0.0,368.45,0.0,0.0,0.0,267646.0,61100.0,0
42538,12000,12000,12000.0,36 months,13.53,407.40,B,B5,10+ years,RENT,...,0.0,0.0,0.0,119.17,0.0,0.0,15386.0,13605.0,8100.0,0
42539,15000,15000,15000.0,36 months,8.90,476.30,A,A5,2 years,MORTGAGE,...,0.0,0.0,0.0,476.30,0.0,0.0,1514.0,272492.0,15400.0,0
42540,12000,12000,12000.0,36 months,11.99,398.52,B,B3,10+ years,MORTGAGE,...,0.0,0.0,0.0,398.52,0.0,0.0,0.0,327264.0,16200.0,0
42541,14000,14000,14000.0,36 months,12.85,470.71,B,B4,4 years,RENT,...,0.0,0.0,0.0,470.71,0.0,0.0,0.0,17672.0,4500.0,0
42542,7550,7550,7550.0,36 months,16.24,266.34,C,C5,3 years,RENT,...,0.0,0.0,0.0,266.34,0.0,0.0,0.0,5759.0,8000.0,0
42543,10000,10000,10000.0,36 months,9.67,321.13,B,B1,7 years,MORTGAGE,...,0.0,0.0,0.0,321.13,0.0,0.0,0.0,39143.0,22300.0,0
42544,3000,3000,3000.0,36 months,12.85,100.87,B,B4,10+ years,RENT,...,0.0,0.0,0.0,2677.23,0.0,0.0,154.0,19530.0,5300.0,0


In [9]:
dataset_c = column_t.undersampling(X=dataset_c, y='status')

In [10]:
x_train = dataset_c.drop("status", axis=1)
y_train = dataset_c["status"]

In [11]:
x_train

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_length,home_ownership,...,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_amnt,collections_12_mths_ex_med,acc_now_delinq,tot_coll_amt,tot_cur_bal,total_rev_hi_lim
336666,15000,15000,15000.0,36 months,6.49,459.67,A,A2,4 years,OWN,...,1093.02,0.0,0.00,0.0000,459.67,0.0,0.0,0.0,38867.0,36200.0
390198,35000,35000,34950.0,60 months,20.99,946.68,E,E4,4 years,MORTGAGE,...,11006.74,0.0,0.00,0.0000,946.68,0.0,0.0,0.0,360928.0,14600.0
268918,9600,9600,9600.0,36 months,8.39,302.56,A,A5,10+ years,RENT,...,781.14,0.0,0.00,0.0000,302.56,0.0,0.0,355.0,22351.0,28200.0
388502,30000,30000,29900.0,36 months,10.15,970.13,B,B2,< 1 year,MORTGAGE,...,2449.39,0.0,0.00,0.0000,22748.09,0.0,0.0,0.0,240300.0,82800.0
199125,20950,20950,20950.0,60 months,18.75,540.58,D,D3,9 years,RENT,...,9767.67,0.0,0.00,0.0000,10175.63,0.0,0.0,60.0,35064.0,34100.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
466251,9000,9000,9000.0,36 months,16.99,320.83,D,D1,10+ years,OWN,...,1847.53,0.0,0.00,0.0000,100.00,0.0,0.0,0.0,21265.0,32000.0
466252,6000,6000,6000.0,36 months,13.53,203.70,B,B5,4 years,MORTGAGE,...,497.25,0.0,682.95,6.8295,203.70,0.0,0.0,0.0,135047.0,20100.0
466276,8525,8525,8525.0,60 months,18.25,217.65,D,D3,5 years,MORTGAGE,...,510.45,0.0,1159.40,11.5940,217.65,0.0,0.0,0.0,116995.0,49800.0
466277,18000,18000,17975.0,36 months,7.90,563.23,A,A4,3 years,RENT,...,1050.78,0.0,0.00,0.0000,563.23,0.0,0.0,0.0,42592.0,50300.0
